In [73]:
import os

import pandas as pd
from pyproj import Proj,transform

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import STAMEN_TONER
from bokeh.models import ColumnDataSource, HoverTool, LabelSet

In [74]:

def convertCoords(row):
    inProj  = Proj(init='epsg:4326')
    outProj = Proj(init='epsg:3857')
    x2,y2 = transform(inProj,outProj,row['stop_lon'],row['stop_lat'])
    return pd.Series({'stop_m-lon':x2,'stop_m-lat':y2})


def plot_stops(network_dir, stop_labels):

    stoptimes_df      = pd.read_csv(os.path.join(network_dir, 'stop_times.txt'), usecols=['trip_id','stop_id','stop_sequence'])
    stops_df          = pd.read_csv(os.path.join(network_dir, 'stops.txt'), usecols=['stop_id','stop_name','stop_lat','stop_lon'])
    xy_merc           = stops_df.apply(convertCoords, axis=1)
    stops_df          = pd.concat([stops_df, xy_merc], axis=1)
    trip_stops_df     = pd.merge(stops_df, stoptimes_df, on='stop_id', how='left')
    trip_count_df     = trip_stops_df.groupby(['stop_id']).size().reset_index(name='trips')
    stops_size_df     = pd.merge(stops_df, trip_count_df, on='stop_id')
    
    size_factor       = stops_size_df['trips'].max()/70
    if stops_size_df['trips'].max() < 10: size_factor = 0.05
       
    source = ColumnDataSource(data=dict(
                         x    =list(stops_size_df['stop_m-lon']), 
                         y    =list(stops_size_df['stop_m-lat']),
                         trips=list(stops_size_df['trips']),
                         size =list(stops_size_df['trips']/size_factor),
                         name =list(stops_size_df['stop_name'])))

    hover = HoverTool(tooltips=[("stop", "@name"),("trips","@trips") ])


    p = figure(x_axis_type="mercator", y_axis_type="mercator")
   # tools=[hover, pan, 'wheel_zoom','save']
    p.add_tile(STAMEN_TONER)
    
    
    p.circle(x = 'x',
         y = 'y',
         source = source,
         size = 'size',
         fill_color="#FF1493",
         line_color="#FF69B4",
         fill_alpha=0.15,
         line_alpha=0.40)
    
    labels = LabelSet(x='x', y='y', text='name', level='glyph',text_color="#FF1493",
              x_offset=5, y_offset=5, source=source, render_mode='canvas')
    
    if stop_labels: p.add_layout(labels)
    
    return p


In [71]:
import ipywidgets as widgets

HOME_DIR = os.path.join(os.getcwd(), '..', "fasttrips", "Examples","networks" )

@widgets.interact(network_dir = os.listdir(HOME_DIR),stop_labels=False)
def plot_net(network_dir,stop_labels):
    p = plot_stops(os.path.join(HOME_DIR,network_dir), stop_labels)
    p.title.text = "Example Network: "+ network_dir
    output_notebook()
    show(p)

interactive(children=(Dropdown(description=u'network_dir', options=('simple', 'bunny_hop', 'psrc_1_1', 'many_bunny_hops'), value='simple'), Checkbox(value=False, description=u'stop_labels'), Output()), _dom_classes=('widget-interact',))